In [2]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import pandas as pd
from ODEDataset import ODEDataset



# Appliances Data

In [3]:
def extract_appliances_data(src: pd.DataFrame) -> pd.DataFrame:
    df = src.copy()
    appliances = ['fan', 'TV', 'refrigerator/freezer', 'food_processor/blender', 'water_heater', 'sterilizer',
                  'printer', 'computer', 'sewing_machine', 'hair_dryer']

    columns = ['ID', 'Respondent_category', 'Type_of_business',
               'Type_of_public_service', 'Street_lightling_presence',
               ]

    for appliance in appliances:
        columns.append(f"m_{appliance}")
        columns.append(f"Presence_{appliance}")
    return df[columns]

In [4]:

from utils.functions import *
from utils.geospatial import *


def add_drived_variables(src: pd.DataFrame) -> pd.DataFrame:
    df = src.copy()
    drived_vars = pd.read_excel("playground/data/CEFA_Tanzania/2016_EE/_master.xlsx", sheet_name="Derived_variables")
    cols = []
    n = len(df)
    m = len(df.columns)
    for index, row in drived_vars.iterrows():

        if "GIS" in row["Formula"] or "data," in row["Formula"]:
            continue
        var = row["All_question"]
        val = eval(row["Formula"])

        cols.append({"var": var, "val": [val] * n})

    for i, col in enumerate(cols):
        df.insert(i + m, col["var"], col["val"])

    return df



In [32]:
def extract_general_and_rename(src: pd.DataFrame) -> pd.DataFrame:
    d = src.copy()
    cols = [
        "Village"
        , "Bill_schedule"
        , "Bill"
        , "Income"
        , "Expenditures"
        , "HH_members"
        , "HH_rooms"
        , "Income_source"]

    return d[cols].rename(
        columns={"Income": "Income_of_household",
                 "Expenditures": "Monthly_expenditure",
                 "HH_members": "People_per_household",
                 "HH_rooms": "Number_of_rooms"
                 },
)

In [33]:
ds = (ODEDataset("CEFA_Tanzania").from_excel(
    "../playground/data/CEFA_Tanzania/2016_EE/Q_EE_HH_Results_General.xlsx")
      .apply(extract_general_and_rename)
      )
# .apply(extract_appliances_data).apply(add_drived_variables))

df = ds.to_dataframe()




In [34]:
df

,Village,Bill_schedule,Bill,Income_of_household,Monthly_expenditure,People_per_household,Number_of_rooms,Income_source
0,Nyombo,M,48000,30000,10000,5,10,Agr
1,Nyombo,M,30000,100000,100000,6,7,Agr
2,Nyombo,M,10000,20000,20000,6,4,Agr
3,Nyombo,M,20000,100000,100000,6,7,Bre
4,Nyombo,M,12000,100000,40000,2,4,Self
5,Nyombo,M,17000,30000,30000,7,7,Agr
6,Nyombo,M,20000,35000,30000,5,8,Agr
7,Nyombo,M,7000,30000,25000,6,3,Agr
8,Nyombo,W,40000,200000,100000,4,8,Agr
9,Nyombo,M,15000,60000,35000,3,7,Agr


In [40]:
x = pd.read_excel("../playground/data/CEFA_Tanzania/2016_EE/Q_EE_HH_Results_Appliances.xlsx")

x.columns

Index(['Q_EE_HH', 'Item', 'Item_power', 'Item_number', 'Time_average',
       'Time_minimum', 'ST_FW1', 'ET_FW1', 'ST_FW2', 'ET_FW2', 'Comments_old',
       'Comments'],
      dtype='object')

In [39]:
for i in x.groupby("Q_EE_HH"):
    print(i[1])
    break

      Q_EE_HH        Item  Item_power  Item_number  Time_average  \
0  Q_EE_HH_01     Charger           5            4           240   
1  Q_EE_HH_01  Ind_lights          18            5           540   
2  Q_EE_HH_01  Out_lights          18            2           540   
3  Q_EE_HH_01       Radio          10            1           180   
4  Q_EE_HH_01          TV          80            1           180   

   Time_minimum  ST_FW1  ET_FW1  ST_FW2  ET_FW2 Comments_old  Comments  
0            60    1080    1260       0       0          NaN       NaN  
1           540       0     360    1080    1440          NaN       NaN  
2           540       0     360    1080    1440          NaN       NaN  
3            10    1080    1260    1440       0          NaN       NaN  
4            10    1080    1260       0       0          NaN       NaN  
